<a href="https://colab.research.google.com/github/Mwadz/Path_description/blob/main/Bart_XML_path_Description_Geneneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Documentation
Here's a much more detailed [explanation](https://docs.google.com/document/d/1EamrnIh1XeeEF-x5uXyurUJi-rHb7bR9DybDn161N6c/edit#) of the notebook

## Loading the Data

In [10]:
import pandas as pd
# reading the data and naming it desc
train = pd.read_csv('/content/train_data.csv')
train.head(2)
print(train.shape)

(480, 2)


In [11]:
eval = pd.read_csv('/content/eval_data.csv')
eval.head(2)
print(eval.shape)

(770, 2)


# Cleaning and Preparing Data

In [12]:
#@title Renaming the columns 
train.columns = ['input_text', 'target_text']
eval.columns = ['input_text', 'target_text']

In [13]:
#@title Checking data
import warnings
warnings.filterwarnings('ignore')
train.head() # replace train with eval or vice versa to check either 

,input_text,target_text
0,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Allows producers and consumers of XML instanc...
1,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,"The ""any attribute"" wildcard is a placeholder..."
2,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,"The name of the city.Changed name to add ""Nam..."
3,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A value from a MISMO prescribed list that spe...
4,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to provide a supp...


In [14]:
# checking for nulls
train.isnull().sum()

input_text     0
target_text    1
dtype: int64

In [16]:
# Dropping the null values
train = train.dropna()
train.isnull().sum()

input_text     0
target_text    0
dtype: int64

In [19]:
# checking for duplicate values
print(train.duplicated().sum())
print(eval.duplicated().sum())

1
2


we'll keep the duplicates since file paths might share desciptions

In [17]:
eval.head()

,input_text,target_text
0,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to describe the s...
1,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Indicates whether the comparable property has...
2,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Specifies the high listing value in the range...
3,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Specifies the low listing value in the range ...
4,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Specifies the total number of comparable prop...


In [20]:
train.head()

,input_text,target_text
0,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Allows producers and consumers of XML instanc...
1,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,"The ""any attribute"" wildcard is a placeholder..."
2,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,"The name of the city.Changed name to add ""Nam..."
3,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A value from a MISMO prescribed list that spe...
4,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to provide a supp...


<font color = orange>**Sidenote**</font>
* In standard practice train data usually makes up 70-80% of the total data while test data takes the rest ie 20-30%
* This should be taken into consideration while evaluating performance

# Modeling

## i) Using Bart (Baseline Model)


In [22]:
# importing all the libraries and class models
import pandas as pd
import gensim
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [23]:
# creating sequencer sequence argument function
model_args = Seq2SeqArgs()
# setting hyperparameters
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True

# Initializing the model
model = Seq2SeqModel(
    encoder_decoder_type="bart", # calling the model type
    encoder_decoder_name="facebook/bart-large", # specific name from huggingface library  
    args=model_args, # initializing set hyperparametes
    use_cuda= True, # should be set to 'TRUE' if using GPU
)

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [24]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    ) 

In [25]:
# Training the model
model.train_model(train, eval_data=eval, matches=count_matches)

  0%|          | 0/479 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/770 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/97 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/770 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/97 [00:00<?, ?it/s]

[' A free-form text field used to describe the source of information for the comparable property.', ' Indicates whether the comparable property has prior sales history.', ' Specifies the high listing value in the range of values within the group of comparable listing properties researched.', ' Specifies the low listing value in the range of values within the group of comparable listing properties researched.', ' Specifies the total number of comparable property listings researched.', ' Specifies the high sale value in the range of values within the group of comparable sales researched.', ' Specifies the low sale value in the range of values within the group of comparable sales researched.', ' Specifies the total number of comparable property sales researched.', ' A free-form text field used to describe or comment on why the sales or transfer history research of the subject property and comparable sales was not performed.', ' Indicates that the sales or transfer history research of the 

(600,
 {'eval_loss': [4.170409465573497,
   0.4153654314193529,
   0.15143361582046316,
   0.14460483028255788,
   0.13272159038706846,
   0.15741201136837302,
   0.14566152659004805,
   0.15200945565995483,
   0.16174346945949436,
   0.16178874007051752],
  'global_step': [60, 120, 180, 240, 300, 360, 420, 480, 540, 600],
  'matches': [0, 0, 0, 2, 3, 3, 16, 12, 19, 18],
  'train_loss': [5.091496467590332,
   0.4631023108959198,
   0.2423015981912613,
   0.049400560557842255,
   0.06771570444107056,
   0.092495396733284,
   0.055861908942461014,
   0.04156510904431343,
   0.0493684820830822,
   0.049730055034160614]})

In [ ]:
# # Evaluating the model
results = model.eval_model(eval)
results


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/125 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.3130427151918411}

In [26]:
# Use the model for prediction
print(model.predict(
        [
            "/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS/ASSET/OWNED_PROPERTY/PROPERTY/IMPROVEMENT/CAR_STORAGES/CAR_STORAGE/AdequateIndicator"
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[' Indicates whether the car is adequate for normal needs.']
